In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Dynamics model

In [ ]:
def dubins_step(x, u, dt):
    """Run one step of dynamics

    Parameters
    ----------
    x : np.array
        State vector (x, y, theta)
    u : np.array
        Control vector (v, w)
    dt : float
        Time step

    Returns
    -------
    np.array
        Updated state vector (x, y, theta)

    """
    x_dot = u[0] * np.cos(x[2])
    y_dot = u[0] * np.sin(x[2])
    theta_dot = u[1]
    x_new = x + np.array([x_dot, y_dot, theta_dot]) * dt
    return x_new

In [ ]:
def dubins_traj(x0, U, dt):
    """Compute dubins trajectory from a sequence of controls
    
    Parameters
    ----------
    x0 : np.array
        Initial state vector (x, y, theta)
    U : np.array
        Control sequence (v, w)
    dt : float
        Time step
    
    Returns
    -------
    np.array
        Trajectory (x, y, theta)
    
    """
    traj = np.zeros((len(U), 3))
    traj[0] = x0
    for i in range(1, len(U)):
        traj[i] = dubins_step(traj[i-1], U[i-1], dt)
    return traj

In [ ]:
DT = 0.1
N = 30
V_MAX = 1  # [m/s]
W_MAX = 1  # [rad/s]
V_BOUNDS = np.array([-V_MAX, V_MAX])
W_BOUNDS = np.array([-W_MAX, W_MAX])

In [ ]:
# Test dubins trajectory
x0 = np.array([0, 0, 0])
U = np.array([[1, 0], [1, 0.1], [1, 0.1], [1, 0.2], [1, 0.2], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0]])
dt = 0.1
traj = dubins_traj(x0, U, dt)
# Plot trajectory
plt.plot(traj[:, 0], traj[:, 1])
plt.show()

In [ ]:
# Quadrotor params
m = 0.5
g = 9.81
I = np.eye(3) * 0.01



In [ ]:
def quadrotor_dynamics(x, u, dt):
    """Run one step of quadrotor dynamics

    Parameters
    ----------
    x : np.array
        State vector (x, y, z, vx, vy, vz, phi, theta, psi, p, q, r)
    u : np.array
        Control vector (F, tau_x, tau_y, tau_z)
    dt : float
        Time step

    Returns
    -------
    np.array
        Updated state vector (x, y, z, vx, vy, vz, phi, theta, psi, p, q, r)

    """
    # Unpack state
    x, y, z, vx, vy, vz, phi, theta, psi, p, q, r = x
    # Unpack control
    F, tau_x, tau_y, tau_z = u
    # Compute acceleration
    ax = (F * np.cos(theta) * np.cos(psi) + tau_x) / params.M
    ay = (F * np.cos(theta) * np.sin(psi) + tau_y) / params.M
    az = (F * np.sin(theta) + tau_z) / params.M
    # Compute angular acceleration
    p_dot = (params.IY - params.IZ) * q * r / params.IX + tau_x / params.IX
    q_dot = (params.IZ - params.IX) * p * r / params.IY + tau_y / params.IY
    r_dot = (params.IX - params.IY) * p * q / params.IZ + tau_z / params.IZ
    # Update state
    x_new = np.array([
        x + vx * dt,
        y + vy * dt,
        z + vz * dt,
        vx + ax * dt,
        vy + ay * dt,
        vz + az * dt,
        phi + p * dt,
        theta + q * dt,
        psi + r * dt,
        p + p_dot * dt,
        q + q_dot * dt,
        r + r_dot * dt,
    ])
    return x_new